# Avoiding For-Loops for Performance

### Task: Count lines in images
Let's consider the following task: We have to count particular lines of 3 dots in images. A pre-processing step gives us matrices indicating where on the image a dot is. Look at the following example to see a single sample. The data is represented by a $5 \times 5$ matrix.

In [3]:
import numpy as np
sample = (np.random.random([5,5])>.5).astype(int)
print(sample)

[[1 0 0 0 1]
 [0 1 0 1 1]
 [1 1 1 1 1]
 [0 0 1 1 1]
 [0 0 0 0 1]]


Let's count the number of diagonals from left to right downwards, E.g. the matrices depicted below have one and three, resp. A line of 4 counts as two possible lines of 3.

```
0 0 0 0 0      1 0 0 0 0 
0 0 1 0 0      0 1 0 1 0
0 0 0 1 0      1 0 1 0 1
0 0 0 0 1      0 1 0 1 0
0 0 0 0 0      0 0 1 0 0 
```

The naive implementation is pretty straight-forward. Try multiple times to really see that it works:

In [4]:
sum = 0
for r in range(3):
    for c in range(3):
        if sample[r][c]==1 and sample[r+1][c+1]==1 and sample[r+2][c+2]== 1:
            sum += 1
print("Found %s line%s" % ((sum, "s") if sum>1 else ("no", "") if sum==0 else (1, "")))

Found 3 lines


And as a function to count from a bunch of samples

In [5]:
def count_num_samples_with_row_of_3_with_forloop(samples):
    sum = 0
    for sample in samples:
        for r in range(3):
            for c in range(3):
                if sample[r][c]==1 and sample[r+1][c+1]==1 and sample[r+2][c+2]== 1:
                    sum+=1
    return sum

In [6]:
samples = [(np.random.random([5,5])>.5).astype(int) for i in range(2000)]
np.shape(samples)

(2000, 5, 5)

In [7]:
%time count_num_samples_with_row_of_3_with_forloop(samples)

Wall time: 14 ms


2277

On my Mac Pro, that takes about 18 microseconds to execute. I promise you: We can do much better!

### Using Vectorization to improve performance

The idea for a vector-based algorithm is the following: We can use element-wise product $\otimes$ (sometimes called *Adamard* product) to detect a row of three - like so:

$$
\left( \begin{array}{ccccc}
1 & 0 & 0 & 0 & 0 \\
0 & 1 & 0 & 1 & 0 \\
0 & 0 & 1 & 0 & 0 \\
0 & 1 & 0 & 0 & 1 \\
0 & 0 & 0 & 0 & 0 
\end{array} \right)
\otimes
\left( \begin{array}{ccccc}
1 & 0 & 0 & 0 & 0 \\
0 & 1 & 0 & 0 & 0 \\
0 & 0 & 1 & 0 & 0 \\
0 & 0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 & 0 
\end{array} \right)
=
\left( \begin{array}{ccccc}
1 & 0 & 0 & 0 & 0 \\
0 & 1 & 0 & 0 & 0 \\
0 & 0 & 1 & 0 & 0 \\
0 & 0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 & 0 
\end{array} \right)
$$

We're essentially masking out all but three dots. Then we count what's left and then if that's exactly three dots, we have detected a line of three.

However, that detects only those lines in the upper left corner. So we need a mask or filter for each of the $9$ possible occurrences of lines of three.

Here's the filter that detects a line in the upper left corner:

In [8]:
filter1 = np.array([[1, 0, 0, 0, 0],
       [0, 1, 0, 0, 0],
       [0, 0, 1, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0]])
print(filter1)

[[1 0 0 0 0]
 [0 1 0 0 0]
 [0 0 1 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]]


#### Shifting with a matrix operator
We can produce the other filters by shifting the line to the right and/or down. We achieve that with a little bit of algebra: Multiplying a flattened $25 \times 1$ representation of the above filter matrix with the below *shifter* matrix of shape $25 \times 25$ produces a filter for a line that's shifted one step to the right

In [11]:
shifter = np.vstack ([np.zeros(25, dtype=int), np.eye(25, dtype=int)])[:-1,:]
print(shifter)

[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 

In [12]:
np.matmul(shifter, filter1.flatten()).reshape([5,5])

array([[0, 1, 0, 0, 0],
       [0, 0, 1, 0, 0],
       [0, 0, 0, 1, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0]])

Ain't that cool?

Note that this approach *wastes* memory, but software architecture is about trade-offs. And we're trading off memory for compute times in this exercise. However, the above calculations are only been done once to prepare the tools for the real work. When it comes to processing the $2000$ samples above, we'll be avoiding every cost in compute time and be careful with memory, too. In reality, there might be millions and billions of records to process. So, this exercise is worth the effort.

### 9 differently positioned filters

In [13]:
filters = np.zeros([25,25], dtype=int)

In the below code, we create an array of filters and call it ```detector```. Each row in the big matrix represents a *flattened* filter for a particular occurence of the diagonal line. Observe that only $9$ of those filters are actually non-zero. That's due to the fact that only $9$ different lines of that kind are possible.

In [14]:
shifter = np.vstack ([np.zeros(25, dtype=int), np.eye(25, dtype=int)])[:-1,:]

def create_detector(f):
    f = f.flatten()
    detector = np.zeros([9,25], dtype=int)

    # These nine shifts produce all our filters
    for i, s in enumerate([0,1,2, 5,6,7, 10,11,12]):
        detector[i] = np.matmul (np.linalg.matrix_power(shifter, s), f)
    return detector

filter1 = np.array([
    [1, 0, 0, 0, 0],
    [0, 1, 0, 0, 0],
    [0, 0, 1, 0, 0],
    [0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0]])        

detector = create_detector(filter1)

print(detector)

[[1 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 1 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 1 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 1 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 1 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 1]]


The below code returns all positions with a count of $3$ as a pair of lists of row indexes and column indexes:

In [15]:
def find_rows_of_3(sample):
    return np.where((np.matmul(detector, sample.flatten()) == 3))[0]

The below code returns all the (*flattened*) positions of diagonals. Try a couple of samples to verify the detector's capabilities.

In [19]:
sample = (np.random.random([5,5])>.5).astype(int)
print(sample)
find_rows_of_3(sample)

[[0 0 1 1 0]
 [0 1 0 0 0]
 [0 1 0 1 1]
 [0 1 1 0 0]
 [0 1 1 1 0]]


array([7], dtype=int64)

### Processing the entire batch

In [20]:
np.shape(samples)

(2000, 5, 5)

In [21]:
samples[0]

array([[1, 0, 1, 0, 0],
       [0, 0, 1, 0, 0],
       [1, 0, 1, 1, 0],
       [1, 0, 0, 1, 0],
       [1, 1, 1, 0, 0]])

In [22]:
def count_num_rows_of_3(samples):
    return np.sum([find_rows_of_3(sample).size for sample in samples])

In [26]:
%time count_num_rows_of_3(samples)

Wall time: 6.93 ms


2277

Depending on your machine it could be a little faster or even slower than the naive implementation. So that's a bit dissapointing. But wait: There's still the loop over the 2000 samples that we can avoid. Let's try that. 

In the below function we're multiplying the entire batch of samples with dimensions $2000 \times 25$ by the detector matrix of dimension $25 \times 25$ in a single go, then count the number of values $3$ in the resulting $2000 \times 25$ matrix

In [27]:
def count_num_samples_with_row_of_3_better(samples):
    all_reshaped = np.reshape(samples, [2000, 25])
    all_checked = np.matmul(detector, np.transpose(all_reshaped))
    return np.sum(all_checked==3)

In [30]:
%time count_num_samples_with_row_of_3_better(samples)

Wall time: 2 ms


2277

Or in an unprofessional fashion - in a single line of code!

In [31]:
np.sum(np.matmul(detector, np.transpose(np.reshape(samples, [2000, 25])))==3)

2277

On my Mac Pro that usually stays significantly below 3ms. That's a massive improvement. Note that the result stems from the fact that there's no longer a forth-and-back between scripting and computation. All computations are basically done in only two steps. The numpy library hands over the entire matrices in a single go to the CPU infrastructure where the hard work is done. 

Note that if you want to get more reliable performance figures you should resort to using ```%timeit``` instead, which calls the function a number of time to come back with reasonably sized statistics. 

In [32]:
%timeit count_num_samples_with_row_of_3_better(samples)

926 µs ± 4.64 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


### Caveats
Vectorized code is much less readable than procedural code. It's a trade-off that we can make easier by providing extensive documentation of what is actually meant by the algebraic expression we're using. Keep that in mind if you want to avoid situations where you find yourself unable to comprehend some code that you've written your self just the day before!

### Want some more?
What you've seen here is indeed not so special. The above matrix multiplication is exactly what a convolutional neural network does. Only it doesn't typically come with fixed parameters but it learns the parameters during training. 

For your own enlightenment you may want to add more filters to also recognize vertical and horizontal lines. 